In [ ]:
print("Hello")

Hello


In [ ]:
!java -version

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [ ]:
!wget q https://dlcdn.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

--2022-05-26 06:22:24--  http://q/
Resolving q (q)... failed: Name or service not known.
wget: unable to resolve host address ‘q’
--2022-05-26 06:22:24--  https://dlcdn.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz.1’

spark-2.4.8-bin-had 100%[===================>] 224.97M   176MB/s    in 1.3s    

2022-05-26 06:22:26 (176 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz.1’ saved [235899716/235899716]

FINISHED --2022-05-26 06:22:26--
Total wall clock time: 1.3s
Downloaded: 1 files, 225M in 1.3s (176 MB/s)


In [ ]:
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
print("spark")

spark


In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 42.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9167adccda95bd64bc135bd5a0305c0033ba7f88138b86133d2dbec5a698b6f2
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
spark.stop()

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
conf = SparkConf()
conf.setMaster("local")
conf.setAppName("testApp")
sc = SparkContext(conf=conf)

In [172]:
data= sc.textFile("/content/sample_data/rejected_2007_to_2018Q4.csv")

In [173]:
data.cache()

/content/sample_data/rejected_2007_to_2018Q4.csv MapPartitionsRDD[225] at textFile at NativeMethodAccessorImpl.java:0

In [174]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [175]:
data.count()

21342776

In [176]:
parData = data.getNumPartitions()

In [177]:
print(parData)

41


In [179]:
rePartitionData=data.repartition(500)

In [188]:
dataSchema= spark.read.csv("/content/sample_data/rejected_2007_to_2018Q4.csv",sep=",", inferSchema=True, header=True)

In [189]:
dataSchema.show(5)

+----------------+-------------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|   Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+-------------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          1000.0|2007-05-26 00:00:00|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|
|          1000.0|2007-05-26 00:00:00|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|
|         11000.0|2007-05-27 00:00:00|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|
|          6000.0|2007-05-27 00:00:00|             waksman|     698.0|              38.64%|   017xx|   MA|         < 1 year|        0.0|
|          1500.0|2007-05-27 00:00:00|   

In [190]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import window
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import datetime
from pyspark.sql.functions import year, month, dayofmonth

In [186]:
tblColData=dataSchema.withColumn('Application Date',F.year(F.to_timestamp('Application Date', 'yyyy')))

In [191]:
tblColData.show(5)

+----------------+----------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|        Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          2000.0|            2016|debt_consolidation|      null|               4.24%|   891xx|   NV|         < 1 year|        0.0|
|          5000.0|            2016|debt_consolidation|      null|                  0%|   918xx|   CA|         < 1 year|        0.0|
|          8775.0|            2016|debt_consolidation|      null|              13.82%|   380xx|   TN|          5 years|        0.0|
|          1000.0|            2016|    small_business|      null|              16.67%|   392xx|   MS|         < 1 year|        0.0|
|         10000.0|            2016|debt_consolidation|      null|           

In [193]:
tblColData1=tblColData.withColumnRenamed("Application Date","ApplicationDate")

In [209]:
dataYear=tblColData1.filter(tblColData1.ApplicationDate >= 2009)

In [210]:
dataYear.show(4)

+----------------+---------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|ApplicationDate|        Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+---------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          8000.0|           2011|       Temp Relief|     771.0|               6.91%|   333xx|   FL|        10+ years|        0.0|
|          2000.0|           2011|debt_consolidation|     589.0|              27.73%|   322xx|   FL|         < 1 year|        0.0|
|         30000.0|           2012|  home_improvement|       0.0|                  0%|   152xx|   PA|         < 1 year|        0.0|
|         12000.0|           2010|       credit_card|     549.0|              32.74%|   773xx|   TX|          7 years|        0.0|
+----------------+---------------+------------------+----------+-------------------

In [211]:
tblColRe=dataYear.withColumnRenamed('ApplicationDate','Application Date')

In [212]:
tblColRe.show(5)

+----------------+----------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|        Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          8000.0|            2011|       Temp Relief|     771.0|               6.91%|   333xx|   FL|        10+ years|        0.0|
|          2000.0|            2011|debt_consolidation|     589.0|              27.73%|   322xx|   FL|         < 1 year|        0.0|
|         30000.0|            2012|  home_improvement|       0.0|                  0%|   152xx|   PA|         < 1 year|        0.0|
|         12000.0|            2010|       credit_card|     549.0|              32.74%|   773xx|   TX|          7 years|        0.0|
|          5800.0|            2016|       credit_card|      null|           

In [213]:

movAvg=tblColRe.withColumn("RiskScoreMA100. ", avg(tblColRe["Risk_Score"]).over(Window.partitionBy("Application Date").rowsBetween(-100,1)) )


In [214]:
movAvg.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+-----------------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code| RiskScoreMA100. |
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+-----------------+
|          2000.0|            2018|               Other|     600.0|                  0%|   154xx|   PA|         < 1 year|        0.0|            600.0|
|          5000.0|            2018|  Debt consolidation|      null|              11.29%|   161xx|   PA|         < 1 year|        0.0|            600.0|
|          6500.0|            2018|  Debt consolidation|      null|               7.63%|   544xx|   WI|          2 years|        0.0|            573.5|
|          5000.0|            2018|    Home improvement|     547.0|              13.09%|

In [219]:
movAvg.write.csv('/content/sample_data/MovingAverage100.csv')